<h1 id="header" align="center">
  Car Price Prediction - Exploratory data analysis.
  <img src="https://media.tenor.com/UtkIFBDTlAAAAAAi/car-cars.gif" width="40px"/>
</h1>

---

### Вступление

_EDA(разведочный анализ данных)_ - это процесс изучения основных свойств данных, нахождения в них закономерностей, аномалий и различных распределений, построений моделей с помощью визуализации.

Известно, что не существует единой последовательности операций для выполнения анализа, но есть **базовые пункты**, которые необходимо выполнить.

Данный разведочный анализ данных проводится в рамках изучения _дисциплины "Инижиниринг управления данными" в университете ИТМО_ и будет проводиться на **выбранном в kaggle dataset**.

### Информация об dataset.

_Dataset - Car Price Prediction Challenge_ - является набором данных об различных проданных автомобилях с их параметрами.  
Например:
1) Manufacturer (Производитель)
2) Leather interior (Кожаный салон)
3) Fuel type (Тип топлива)
4) ...

Всего таких параметров - **15**.

Ссылка на оригинальный dataset - https://www.kaggle.com/datasets/deepcontractor/car-price-prediction-challenge?resource=download  
Ссылка на dataset, сохраненный в Google Drive - https://docs.google.com/spreadsheets/d/1PMhtD3LqyCzlZMEh-8aDPxre0wPw8v0U/edit?usp=drive_link&ouid=100105970921534140705&rtpof=true&sd=true


### Цель (target) EDA.

Целью текущего EDA является -  __выявление ключевых факторов (features), влияющих на стоимость__.  
  
Кроме того передо мной стоит цель соответствовать следующим __метрикам__ (как минимум первым двум):

![Metrics](./images/Metrics.png)

---

### Этапы EDA.

1) Загрузка данных с сайта и сохранения их в директории для дальнейшего взаимодействия

>[!IMPORTANT]  
>Не забываем добавить в нашего окружение (Conda+Poetry) необходимые зависимости:
>1) _pandas_
>2) _numpy_
>3) _seaborn_
>4) _matplotlib_
>5) _openpyxl_
>Всегда активируем окружение как рассказано в файле [README.md](../README.md).

Первостепенно, так как наш датасет лежит не в директории проекта, необходимо выполнить создание папки data в корневой директории. А затем выполнить загрузку туда "сырых данных" в формате xlsx.  
Также в данном пункте выполним подключение основных библиотек для проведения дальнейшего EDA.







In [2]:
import os
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)

FILE_ID = "1PMhtD3LqyCzlZMEh-8aDPxre0wPw8v0U"  # ID файла на Google Drive
file_url = f"https://drive.google.com/uc?id={FILE_ID}"

pd.set_option("display.max_columns", None) # убирает ограничения отображения dataset

data_dir = "../data" # создаем директорию data в директории notebooks.
if not os.path.exists(data_dir):
    os.makedirs(data_dir) 

data_path = os.path.join(data_dir, "data_car.xlsx") # путь до файла dataset

if os.path.isfile(data_path): # если файл есть в data - читаем, если нет - скачиваем
    df = pd.read_excel(data_path)
    print(f"{'-'*60}")
    print ("Dataset прочитан из директории data...")
    print(f"{'-'*60}")
else:
    response = requests.get(file_url)
    f_path = os.path.join(data_dir, "data_car.xlsx")
    with open(f_path, "wb") as f:
        f.write(response.content)

    df = pd.read_excel(f_path)
    print(f"{'-'*60}")
    print("Dataset загружен в директорию data и прочитан...")
    print(f"{'-'*60}")


------------------------------------------------------------
Dataset загружен в директорию data и прочитан...
------------------------------------------------------------


2) Первичный осмотр данных

После чтения/загрузки dataset в директории необходимо посмотреть, что было загружено.  
Кроме того необходимо первично оценить данные, которыми наполнен файл xlsx.  
Поэтому выведем 10 первых и 10 последних значений dataset.

In [3]:
df.head(10)        

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
5,45802912,39493,891,HYUNDAI,Santa FE,2016,Jeep,Yes,Diesel,2,160931 km,4.0,Automatic,Front,04-May,Left wheel,White,4
6,45656768,1803,761,TOYOTA,Prius,2010,Hatchback,Yes,Hybrid,1.8,258909 km,4.0,Automatic,Front,04-May,Left wheel,White,12
7,45816158,549,751,HYUNDAI,Sonata,2013,Sedan,Yes,Petrol,2.4,216118 km,4.0,Automatic,Front,04-May,Left wheel,Grey,12
8,45641395,1098,394,TOYOTA,Camry,2014,Sedan,Yes,Hybrid,2.5,398069 km,4.0,Automatic,Front,04-May,Left wheel,Black,12
9,45756839,26657,-,LEXUS,RX 350,2007,Jeep,Yes,Petrol,3.5,128500 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12


In [4]:
df.tail(10)

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
19227,45769427,29793,1053,MERCEDES-BENZ,E 350,2014,Sedan,Yes,Diesel,3.5,219030 km,6.0,Automatic,4x4,04-May,Left wheel,Black,12
19228,45773726,706,1850,MERCEDES-BENZ,E 350,2008,Sedan,Yes,Diesel,3.5,122874 km,6.0,Automatic,Rear,04-May,Left wheel,Black,12
19229,39977395,50,-,TOYOTA,Prius,2008,Hatchback,No,Hybrid,1.5,150000 km,4.0,Automatic,Front,04-May,Left wheel,Silver,6
19230,45760891,470,645,TOYOTA,Prius,2011,Hatchback,Yes,Hybrid,1.8,307325 km,4.0,Automatic,Front,04-May,Left wheel,Silver,12
19231,45772306,5802,1055,MERCEDES-BENZ,E 350,2013,Sedan,Yes,Diesel,3.5,107800 km,6.0,Automatic,Rear,04-May,Left wheel,Grey,12
19232,45798355,8467,-,MERCEDES-BENZ,CLK 200,1999,Coupe,Yes,CNG,2.0 Turbo,300000 km,4.0,Manual,Rear,02-Mar,Left wheel,Silver,5
19233,45778856,15681,831,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,161600 km,4.0,Tiptronic,Front,04-May,Left wheel,Red,8
19234,45804997,26108,836,HYUNDAI,Tucson,2010,Jeep,Yes,Diesel,2,116365 km,4.0,Automatic,Front,04-May,Left wheel,Grey,4
19235,45793526,5331,1288,CHEVROLET,Captiva,2007,Jeep,Yes,Diesel,2,51258 km,4.0,Automatic,Front,04-May,Left wheel,Black,4
19236,45813273,470,753,HYUNDAI,Sonata,2012,Sedan,Yes,Hybrid,2.4,186923 km,4.0,Automatic,Front,04-May,Left wheel,White,12
